<a href="https://colab.research.google.com/github/OlesiaPoliakova/Education/blob/main/PyTestFixtures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![alt text](https://upload.wikimedia.org/wikipedia/commons/c/c3/Python-logo-notext.svg)      PYTEST BASICS

## ![fixture](https://images.beautifulhalo.com/images/128x128/201710/P/1507634075742.jpg) PYTEST FIXTURES

In [ ]:
#@title Installing dependencies

!pip install pytest ipython_pytest ipyext unittest2
%load_ext ipython_pytest
%load_ext ipyext.writeandexecute
!rm -rf work_imports
!mkdir work_imports
!touch work_imports/__init__.py

## ![bank acount](https://cdn0.iconfinder.com/data/icons/banking-color/64/open-account-bank-passbook-128.png) Bank Account Management App

###Card Kinds

In [ ]:
%%writeandexecute -i identifier work_imports/card_kinds.py
from enum import Enum


class CardsKinds(Enum):
    NO_CARD = "No Card"
    NORMAL_CARD = "Normal Card"
    GOLD_CARD = "Gold Card"
    PLATINUM_CARD = "Platinum Card"

### Custom Exceptions

In [ ]:
%%writeandexecute -i identifier work_imports/exceptions.py

class IncorrectDebetError(Exception):
    pass

class TakeMoneyUnavailabilityError(Exception):
    pass

### Main Programm Logic

In [ ]:
%%writeandexecute -i identifier work_imports/bank_support.py
from random import randint
from work_imports.card_kinds import CardsKinds
from work_imports.exceptions import *

class BankSupport(object):
    def __init__(self, account_debet=0.0):
        self.account_id = randint(10000, 99999)
        self.account_debet = account_debet

    @property
    def card_kind(self):
        if self.account_debet < 0.0:
            raise IncorrectDebetError
            (
                f"Bank account #{str(self.account_id)}. \
                Debet is under zero: {str(self.account_debet)}"
            )
        elif self.account_debet == 0.0:
            return CardsKinds.NO_CARD
        elif self.account_debet > 0.0 and \
        self.account_debet <= 9000.0:
            return CardsKinds.NORMAL_CARD
        elif self.account_debet > 10000.0 and \
        self.account_debet <= 99999.0:
            return CardsKinds.GOLD_CARD
        return CardsKinds.PLATINUM_CARD

    def charge_account(self, sum: float):
        if self.account_debet < 0.0:
            raise IncorrectDebetError
            (
                f"Bank account #{str(self.account_id)}. \
                Debet is under zero: {str(self.account_debet)}"
            )

    def take_founds(self, sum: float):
        if self.account_debet < 0.0:
            raise IncorrectDebetError
            (
                f"Bank account #{str(self.account_id)}. \
                Debet is under zero: {str(self.account_debet)}"
            )
        elif self.account_debet < sum:
            raise TakeMoneyUnavailabilityError
            (
                f"Bank account #{str(self.account_id)}. \
                Sum to take {str(sum)} is more than your debet: \
                {str(self.account_debet)}"
            )
        self.account_debet -= sum

### PyTest Suite with the fixtures

In [ ]:
%%pytest
import pytest
from work_imports.bank_support import BankSupport
from work_imports.card_kinds import CardsKinds
from work_imports.exceptions import *


@pytest.fixture
def check_exception():
    def checker(function, exception: Exception, *args):
        try:
            function(*args)
        except exception:
            return True
        except Exception:
            return False
        return False
    return checker

@pytest.fixture
def account_provider():
    return BankSupport

@pytest.fixture
def card_provider():
    return CardsKinds

def test_no_card(account_provider, card_provider):
    card_kind = account_provider(0).card_kind
    assert card_kind == card_provider.NO_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {card_provider.NO_CARD}"

def test_normal_card(account_provider, card_provider):
    card_kind = account_provider(1).card_kind
    assert card_kind == card_provider.NORMAL_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {card_provider.NORMAL_CARD}"

def test_gold_card(account_provider, card_provider):
    card_kind = account_provider(10001).card_kind
    assert card_kind == card_provider.GOLD_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {card_provider.GOLD_CARD}"

def test_platinum(account_provider, card_provider):
    card_kind = account_provider(10001).card_kind
    assert card_kind == card_provider.GOLD_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {card_provider.GOLD_CARD}"

def test_negative_debet(account_provider, check_exception):
    negative_debet = -10.0
    assert check_exception(
        account_provider, 
        IncorrectDebetError, 
        negative_debet), \
    f"Exception IncorrectDebetError hasn't been raised \
    when created account \
    with negative debet {str(negative_debet)}"

def test_take_more_then(account_provider, check_exception):
    assert check_exception(
        account_provider(100).take_founds, 
        TakeMoneyUnavailabilityError, 101), \
    "Exception TakeMoneyUnavailabilityError hasn't been raised \
    when taken funds more then money is on the debet"

### PyTest tests with scoped fixtures

In [ ]:
%%pytest
import pytest
from work_imports.bank_support import BankSupport
from work_imports.card_kinds import CardsKinds
from work_imports.exceptions import *

#pytest -v --setup-show 
#--debug test/pytest/TestBankSupportWithFixtureScopeTuning.py

class Config(object):
    card_provider = None
    account_provider = None

@pytest.fixture(scope="function")
def check_exception():
    def checker(function, exception: Exception, *args):
        try:
            print
            (
                f"Checking for exception {exception} \
                for function {function.__name__}"
            )
            
            function(*args)
            
        except exception:
            return True
        
        except Exception:
            return False
        
        return False
    
    return checker

@pytest.fixture(scope="session", autouse=True)
def account_provider():
    print(f"Providing Banking Account {BankSupport}")
    Config.account_provider = BankSupport

@pytest.fixture(scope="module", autouse=True)
def card_provider():
    print("Providing Card types enumeration")
    Config.card_provider = CardsKinds

def test_no_card():
    card_kind = Config.account_provider(0).card_kind
    assert card_kind == Config.card_provider.NO_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {Config.card_provider.NO_CARD}"

def test_normal_card():
    card_kind = Config.account_provider(1).card_kind
    assert card_kind == Config.card_provider.NORMAL_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {Config.card_provider.NORMAL_CARD}"

def test_gold_card():
    card_kind = Config.account_provider(10001).card_kind
    assert card_kind == Config.card_provider.GOLD_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {Config.card_provider.GOLD_CARD}"

def test_platinum():
    card_kind = Config.account_provider(10001).card_kind
    assert card_kind == Config.card_provider.GOLD_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {Config.card_provider.GOLD_CARD}"

def test_negative_debet(check_exception):
    negative_debet = -10.0
    assert check_exception
    (
        Config.account_provider, 
        IncorrectDebetError, 
        negative_debet
    ), \
    f"Exception IncorrectDebetError hasn't been raised when \
    created account with negative debet {str(negative_debet)}"

def test_take_more_then(check_exception):
    assert check_exception
    (
        Config.account_provider(100).take_founds, 
        TakeMoneyUnavailabilityError, 101), \
    f"Exception TakeMoneyUnavailabilityError hasn't been raised \
    when taken funds more then money is on the debet"


### PyTest tests with teardowns

In [ ]:
%%pytest
import pytest
from work_imports.bank_support import BankSupport
from work_imports.card_kinds import CardsKinds
from work_imports.exceptions import *

#pytest -v --setup-show 
#--debug test/pytest/TestBankSupportWithFixtureScopeTuning.py
class Config(object):
    card_provider = None
    account_provider = None
    
@pytest.fixture(scope="function")
def check_exception():
    def checker(function, exception: Exception, *args):
        try:
            print
            (
                f"Checking for exception {exception} \
                for function {function.__name__}"
            )
            
            function(*args)
            
        except exception:
            return True
        
        except Exception:
            return False
        
        return False
    
    return checker

@pytest.fixture(scope="session", autouse=True)
def account_provider(request):
    print(f"Providing Banking Account {BankSupport}")
    Config.account_provider = BankSupport
    
    def teardown():
      print("Deleting bank account")
      del Config.account_provider
    
    request.addfinalizer(teardown)

def delete_cards():
  print("Deleting card kinds")
  del Config.card_provider


@pytest.fixture(scope="session", autouse=True)
def card_provider():
    print("Providing Card types enumeration")
    Config.card_provider = CardsKinds
    yield
    delete_cards()

def test_no_card():
    card_kind = Config.account_provider(0).card_kind
    assert card_kind == Config.card_provider.NO_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {Config.card_provider.NO_CARD}"

def test_normal_card():
    card_kind = Config.account_provider(1).card_kind
    assert card_kind == Config.card_provider.NORMAL_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {Config.card_provider.NORMAL_CARD}"

def test_gold_card():
    card_kind = Config.account_provider(10001).card_kind
    assert card_kind == Config.card_provider.GOLD_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {Config.card_provider.GOLD_CARD}"

def test_platinum():
    card_kind = Config.account_provider(10001).card_kind
    assert card_kind == Config.card_provider.GOLD_CARD, \
    f"Incorrect type of card has been returned: {card_kind}. \
    Should be {Config.card_provider.GOLD_CARD}"

def test_negative_debet(check_exception):
    negative_debet = -10.0
    assert check_exception
    (
        Config.account_provider, 
        IncorrectDebetError, 
        negative_debet
    ), \
    f"Exception IncorrectDebetError hasn't been raised when \
    created account with negative debet {str(negative_debet)}"

def test_take_more_then(check_exception):
    assert check_exception
    (
        Config.account_provider(100).take_founds, 
        TakeMoneyUnavailabilityError, 101), \
    "Exception TakeMoneyUnavailabilityError hasn't been raised \
    when taken funds more then money is on the debet"
